## Exercise 103
> Let $\mathbb{F}$ be the field $\mathbb{F}_5$ of modular 5 arithmetics from example 16. Brain-compile the following PAPER statement into an algebraic circuit:
> 
> ```plaintext
> statement STUPID_CIRC {F: F_5} {
>   
>   fn foo(in_1 : F, in_2 : F)->(out_1 : F, out_2 : F,){
>     constant c_1 : F = 3 ;
>     out_1<== ADD( MUL( c_1 , in_1 ) , in_1 ) ;
>     out_2<== INV( c_1 , in_2 ) ; };
>   
>   fn main(in_1 : F, in_2 ; F)->(out_1 : F, out_2 : TYPE_2){
>     constant (c_1,c_2) : (F,F) = (3,2) ;
>     (out_1,out_2) <== foo(in_1, in_2) ; };
> }
> ```

```mermaid
flowchart TD
  %% inputs & constants

  subgraph inputs
    in_1["in_1 "]
    in_2["in_2"]
  end

  %% gates
  m1((*))
  m2((*))
  a1((+))

  %% wirings
  in_1 --> a1 --> STUPID_CIRC
  in_1 --> m1 --> a1
  in_2 --> m2 
  in_2 --> STUPID_CIRC
  3 --> m1
  3 --> m2 --1--> 1

  %% outputs
  subgraph outputs
    STUPID_CIRC["STUPID_CIRC"]
  end

```

## Exercise 104
> Consider the tiny-jubjub curve from example 71 and its associated circuit 124. Write a statement in PAPER that brain-compiles the statement into a circuit equivalent to the one derived in 124, assuming that the curve point is the instance and every other assignment is a witness.

```rust
statement TINY_JUBJUB {F: F_13} {

  fn tiny_jubjub_point(x : F, y : F)->(out_1 : F, out_2 : F,){
    constant (a, d) : F = (3, 8) ;
    field x2 <== MUL ( x , x );
    field y2 <== MUL ( y , y );
    
    w1 <== a*x2 + y2;
    w2 <=== 1 + d*x2*y2;
    w1 - w2 === 0;
  };
    
  fn main(public x : F, public y ; F)->(out_1 : F, out_2 : F){
    tiny_jubjub_point(x, y) ; 
  };
}
```

## Exercise 105
>Let  $\mathbb{F} = \mathbb{F}_{13}$ be the modular 13 prime field and $x \in \mathbb{F}$ some field element. Define a statement in PAPER such that given instance $x$, a field element $y \in \mathbb{F}$ is a witness for the statement if and only if $y$ is the square root of $ x$.
>
>Brain-compile the statement into a circuit and derive its associated Rank-1 Constraint System. Consider the instance $x = 9$ and compute a constructive proof for the statement.

```rust
statement SQARE_ROOT {F: F_13} {

  fn square_root(x : F, y : F)->(out_1 : F, out_2 : F,){
    x === MUL ( y , y );
  };
    
  fn main(public x : F, y ; F) {
    square_root(x, y) ; 
  };
}
```

The associated R1CS program:

$$
    \begin{align}
        W_1 \cdot W_1 &= I_1
    \end{align}
$$

The constructive proof $\pi = \langle 9 ; 3 \rangle$  and $\langle 9 ; 4 \rangle$



## Exercise 106
> Let $\mathbb{F}$ be a finite field and let $b_1$ as well as $b_2$ be two boolean constrained variables from $\mathbb{F}$. Show that the equation $ \text{OR}(b_1, b_2) = 1 - (1 - b_1) \cdot (1 - b_2) $ holds true.
>
> Use this equation to derive an algebraic circuit with ingoing variables $b_1$ and $b_2$ and outgoing variable $\text{OR}(b_1, b_2)$ such that $b_1$ and $b_2$ are boolean constrained and the circuit has a valid assignment if and only if $\text{OR}(b_1, b_2) = b_1 \lor b_2$.
>
> Use the technique from 6.2.2.4 to transform this circuit into a Rank-1 Constraint System and find its full solution set. Define a PAPER function that brain-compiles into the circuit.

1. $\text{OR}(b_1, b_2) = 1 - (1 - b_1) \cdot (1 - b_2) \rightarrow \text{ NOT}(\text{ NOT}(b_1) \land \text{ NOT}(b_2)) \rightarrow \text{ NOT}(\text{ NOT}(b_1)) \lor \text{ NOT}(\text{ NOT}(b_2))\rightarrow b_1 \lor b_2$

2. The algebraic circuit
```mermaid
flowchart TD
  %% inputs & constants

  subgraph inputs
    b1["b_1 "]
    b2["b_2"]
  end

  %% gates
  m1((*))
  m2((*))
  m3((*))
  m4((*))
  a1((+))
  a2((+))
  a3((+))

  %% wirings
  -1 --> m1 
  b1 --> m1 --> a1
  1 --> a1 --> m3
  -1 --> m2
  b2 --> m2 --> a2
  1 --> a2 --> m3 --> m4
  -1 --> m4 --> a3
  1 --> a3 --> or

  %% outputs
  subgraph outputs
    or["OR"]
  end
```

3. R1CS:

$$
    \begin{align}
        (1 + -1 \cdot I_1) \cdot (1 + -1 \cdot I_2) &= W_1 \\
        (1 - W_1) \cdot 1 &= 1
    \end{align}
$$

The full set solution is given by $SOL_\pi = \{ \langle 1, 0 ; 0 \rangle, \langle 0, 1 ; 0 \rangle, \langle 1, 1 ; 0 \rangle \}$



## Exercise 107. 
> Let $\mathbb{F}$ be a finite field. Derive algebraic circuits and associated Rank-1 Constraint Systems for the following operators: NOR, XOR, NAND, EQU.

1. $\text{ NOR}(b_1, b_2) \rightarrow \text{ NOT}(\text{ OR}(b_1, b_2)) \rightarrow \text{ NOT}(b_1) \land \text{ NOT}(b_2)$

```mermaid
flowchart LR
  %% inputs & constants
  subgraph inputs
    b1["b_1 "]
    b2["b_2"]
  end

  %% gates
  n1(NOT)
  n2(NOT)
  land1(AND)

  %% wirings
  b1 --> n1
  b2 --> n2
  n1 & n2 --> land1 --> nor

  %% outputs
  subgraph outputs
    nor["NOR"]
  end
```

    R1CS:

$$
    \begin{align}
        (1 - I_1) \cdot (1 - I_2) &= W_1 
    \end{align}
$$
    
2. $\text{XOR}(b_1, b_2) \rightarrow \text{ OR}(b_1, b_2) - \text{ AND}(b_1, b_2) \rightarrow 1 - (1 - b_1) \cdot (1 - b_2) - b_1 \cdot b_2 \rightarrow 1 - 1 + b_1 + b_2 - b_1 \cdot b_2 - b_1 \cdot b_2 \rightarrow b_1 + b_2 - 2 \cdot b_1 \cdot b_2$

```mermaid
flowchart LR
  %% inputs & constants
  2
  -1

  subgraph inputs
    b1["b_1 "]
    b2["b_2"]
  end

  %% gates
  a1((+))
  a2((+))
  m1((*))
  m2((*))
  m3((*))
  

  %% wirings
  b1 & b2 --> a1 & m1
  m1 & 2 --> m2
  m2 & -1 --> m3
  a1 & m3 --> a2 --> xor

  %% outputs
  subgraph outputs
    xor["XOR"]
  end
```

    R1CS:

$$
    \begin{align}
         -2 \cdot I_1 \cdot I_2 &= W_1 \\
        1 \cdot (I_1 + I_2 + W1) &= W_2
    \end{align}
$$

3. $\text{ NAND}(b_1, b_2) \rightarrow \text{ NOT}(\text{ AND}(b_1, b_2)) \rightarrow \text{ NOT}(b_1) \lor \text{ NOT}(b_1) \rightarrow 1 - (1 - (1 - b_1)) \cdot (1 - (1 - b_2)) \rightarrow 1 - b_1 \cdot b2$

```mermaid
flowchart LR
  %% inputs & constants

  subgraph inputs
    b1["b_1 "]
    b2["b_2"]
  end

  %% gates
  n1(NOT)
  m1((*))
  
  %% wirings
  b1 & b2 --> m1 --> n1 --> nand

  %% outputs
  subgraph outputs
    nand["NAND"]
  end
```

R1CS:
$$
\begin{align}
        I_1 \cdot I_2 &= W_1 \\
        1 \cdot (1 - W_1) &= W_2
\end{align}
$$

4. $\text{ EQU}(b_1, b_2) \rightarrow b_1 - b_2 = 0 $ (from [circom-lib](https://github.com/iden3/circomlib/blob/master/circuits/comparators.circom#L37C10-L37C17)

```mermaid
flowchart LR
  %% inputs & constants
  -1

  subgraph inputs
    b1["b_1 "]
    b2["b_2"]
  end

  %% gates
  a1((+))
  m1((*))
  
  %% wirings
  -1 & b2 --> m1 
  b1 & m1 --> a1 --> eq

  %% outputs
  subgraph outputs
    eq["EQU"]
  end
```

R1CS:
$$
\begin{align}
        I_1 - I_2 &= 0 \\
\end{align}
$$



## Exercise 108
> Let $\mathbb{F} = \mathbb{F}_{13}$ and $N = 4$ be fixed and let $x$ be of $u_N$ type. Define circuits and associated R1CS for the left and right bit-shift operators $x << 2$ as well as $x >> 2$. Execute the associated circuit for $x : u_4 = 11$ and generate a constructive proof for R1CS satisfiability.

```mermaid
flowchart TD
  %% inputs & constants
  0
  
  subgraph inputs
    subgraph x
      x1["x_1"]
      x2["x_2"]
      x3["x_3"]
      x4["x_4"]
    end
  end

  %% wirings
  0 --> y1
  0 --> y2
  x1 --> y3
  x2 --> y4

  %% outputs
  subgraph outputs
    subgraph y
      y1["y_1"]
      y2["y_2"]
      y3["y_3"]
      y4["y_4"]
    end
  end
```

## Exercise 109
> Let $\mathbb{F} = \mathbb{F}_{13}$ and $N = 2$ be fixed. Define a circuit and associated R1CS for the addition operator $\text{ADD} : u_N \times u_N \to u_N$. Execute the associated circuit to compute $\text{ADD}(2,7)$ for $2,7 \in u_N$.

```mermaid
flowchart TD
  %% inputs & constants
  subgraph inputs
    subgraph x
      x1["x_1"]
      x2["x_2"]
    end

    subgraph y
      y1["y_1"]
      y2["y_2"]
    end
  end

  %% gates
  m1(XOR)
  m2(XOR)
  a1(AND)
  a2(XOR)

  %% wirings
  x1 --> m1
  y1 --> m1 --> o1
  x1 --> a1
  y1 --> a1 --> a2
  x2 --> m2
  y2 --> m2 --> a2 --> o2

  %% outputs
  subgraph outputs
    subgraph out
      o1["out_1"]
      o2["out_2"]
    end
  end
```

## Exercise 110
> Execute the setup phase for the following PAPER code (That is brain compile the code into a circuit and derive the associated R1CS).
>
> ```
> statement MASK_MERGE {F: F_{5}, N: 4} {
>   fn main(pub a : uN, pub b : uN) -> uN {
>     let constant mask : uN = 10;
>     let r : uN;
>     r <== XOR(a, AND(XOR(a, b), mask));
>     return r;
>   }
> }
> ```
>
> Let $L_{\text{mask_merge}}$ be the language defined by the circuit. Provide a constructive knowledge proof in $L_{\text{mask_merge}}$ for the instance $I = (I_a, I_b) = (14, 7)$.

```mermaid
flowchart TD
  %% inputs & constants
  subgraph inputs
    direction LR
    subgraph a
      a4["a_4"]
      a3["a_3"]
      a2["a_2"]
      a1["a_1"]
    end

    subgraph b
      b4["b_4"]
      b3["b_3"]
      b2["b_2"]
      b1["b_1"]
    end
  end

  subgraph mask
      m4["1"]
      m3["0"]
      m2["1"]
      m1["0"]
  end

  %% gates
  xor1(XOR)
  xor2(XOR)
  land1(AND)

  %% wirings
  a & b --> xor1 --> land1
  mask --> land1
  land1 & a --> xor2 --> out

  %% outputs
  subgraph out
      out1
      out2
      out3
      out4
  end
```

$$
\begin{align*}
[ 1, 1, 1, 0 ] \oplus [ 0, 1, 1, 1 ] &= [ 1, 0, 0, 1 ] \\
[ 1, 0, 0, 1 ] \land  [ 1, 0, 1, 0 ] &= [ 1, 0, 0, 0 ] \\
[ 0, 1, 1, 1 ] \oplus [ 1, 0, 0, 0 ] &= [ 1, 1, 1, 1 ]
\end{align*}
$$


## Exercise 111. 
> Write the circuit and associated Rank-1 Constraint System for a Weierstrass curve of a given field $\mathbb{F}$.

The Weierstrauss Curve is defined as:
$$
\begin{align*}
    y^2 &= x^3 + a\cdot x + b
\end{align*}
$$

The circuit:

```mermaid
flowchart TD
  %% inputs & constants
  -1
  a 
  b
  
  subgraph inputs
    direction LR
    x
    y
  end

  %% gates
  m1((*))
  m2((*))
  m3((*))
  m4((*))
  m5((*))
  a1((+))
  a2((+))
  a3((+))

  %% wirings
  a --> m1
  x --> m1
  m1 --> a1
  b --> a1

  x --> m2
  m2 --> m3
  x --> m3

  a1 --> a2
  m3 --> a2
  
  y --> m4
  m4 --> m5
  -1 --> m5

  a2 --> a3
  m5 --> a3
  a3 --=0--> ECC

  %% outputs
  ECC
```

The R1CS: $I_1 \rightarrow x$ and $I_2 \rightarrow y$

$$
\begin{align*}
    x \cdot x &= W_1 \\
    x \cdot W_2 &= W_2 \\
    y \cdot y &= W_3 \\
    1 \cdot (W2 + a \cdot x + b - W3) &= 0
\end{align*}
$$


## Exercise 112. 
> Let $\mathbb{F}$ be a field. Define a circuit that enforces field inversion for a point of a twisted Edwards curve over $\mathbb{F}$.

The inverse of a point $(x, y)$ in the Twisted Edwards Curve is defined as: $(-x, y)$. The the circuit is:

```mermaid
flowchart LR
  %% inputs & constants
  -1
  
  subgraph inputs
    direction LR
    x
    y
  end

  %% gates
  te(TWISTED_EDWARDS)
  m1((*))

  %% wirings
  x & -1 --> m1
  y & m1 --> INV
  x & y --> te

  %% outputs
  subgraph outputs
    INV
  end
```

## Exercise 113.
> Write the circuit and associated Rank-1 Constraint System for a Weierstrass addition law of a given field $\mathbb{F}$.
